# Creative Testing Significance Testing

### Prerequisites
1. Creative testing data should be in long form (i.e. each row being a respondent and each column being a question)
2. Have the data in either CSV format or gsheets format

### Import libraries

Just run these.

In [50]:
import re
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest as ztest
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from authentication.authenticator import Authenticator
from sheets.sheetmanager import SheetManager

In [51]:
# TODO: improve accuracy of decimals etc.

results_dp = 2
results_percentage_dp = 2

data_url = 'https://docs.google.com/spreadsheets/d/1Yr-jmDKJixCcRRBK73yaWU-VtxZlcW5bnwm8wHqm538/edit#gid=1662096454'
data_sheet_name = 'Data'
results_url = 'https://docs.google.com/spreadsheets/d/1Yr-jmDKJixCcRRBK73yaWU-VtxZlcW5bnwm8wHqm538/edit#gid=1066786955'
results_sheet_name = 'Tests'



In [52]:
csv_in = False
csv_out = False

def get_csv_in(CSV_IN):
    csv_in = CSV_IN
    if csv_in:
        print("Notebook will take in CSV data")
    else:
        print("Notebook will take in gsheets data")
def get_csv_out(CSV_OUT):
    csv_out = CSV_OUT
    if csv_out:
        print("Notebook will save results to a CSV")
    else:
        print("Notebook will update results to a gsheet")
'''        
def set_data_url(url):
    try:
        data_spreadsheetId = re.findall(r"/spreadsheets/d/([a-zA-Z0-9-_]+)", url)[0]
    except IndexError:
        data_spreadsheetId = None
        print('Please make sure the URL is correct')
    print('Data URL set to', url)
    if data_spreadsheetId != None:
        print('Spreadsheet ID detected as:', data_spreadsheetId)
    
def set_data_sheet_name(name):
    data_data_range = name
    print('Data Sheet Name set to', name)
    
def set_results_url(url):
    results_spreadsheetId = re.findall(r"/spreadsheets/d/([a-zA-Z0-9-_]+)", url)[0]
    print('Results URL set to', url)
    
def set_results_sheet_name(name):
    results_data_range = name
    print('Results Sheet Name set to', name)
'''

'        \ndef set_data_url(url):\n    try:\n        data_spreadsheetId = re.findall(r"/spreadsheets/d/([a-zA-Z0-9-_]+)", url)[0]\n    except IndexError:\n        data_spreadsheetId = None\n        print(\'Please make sure the URL is correct\')\n    print(\'Data URL set to\', url)\n    if data_spreadsheetId != None:\n        print(\'Spreadsheet ID detected as:\', data_spreadsheetId)\n    \ndef set_data_sheet_name(name):\n    data_data_range = name\n    print(\'Data Sheet Name set to\', name)\n    \ndef set_results_url(url):\n    results_spreadsheetId = re.findall(r"/spreadsheets/d/([a-zA-Z0-9-_]+)", url)[0]\n    print(\'Results URL set to\', url)\n    \ndef set_results_sheet_name(name):\n    results_data_range = name\n    print(\'Results Sheet Name set to\', name)\n'

### Options Menu

1. `csv_in`: `True` or `False`. 
<br>True to read in a csv. 
<br>False to read in a gsheet.
<br>--
2. `csv_out`: `True` or `False`. 
<br>True to output results to a csv. 
<br>False to output results to a gsheet.
<br>--
3. `results_dp`: `int`.
<br>Indicate the number of decimal places for non-percentage results
<br>--
4. `results_percentage_dp`: `int`.
<br>Indicate the number of decimal places for percentage results (e.g. Abs Lift %)

In [53]:
interact(get_csv_in, CSV_IN=False)
interact(get_csv_out, CSV_OUT=False)

#interact(set_data_url, url='Data GSheet URL')
#interact(set_data_sheet_name, name='Data Sheet Name')
print()

interactive(children=(Checkbox(value=False, description='CSV_IN'), Output()), _dom_classes=('widget-interact',…

interactive(children=(Checkbox(value=False, description='CSV_OUT'), Output()), _dom_classes=('widget-interact'…

### Initialize necessary strings for gsheets
__Necessary strings__ if either `csv_in` or `csv_out` is `False`.
1. `keys`
<br>--
2. `SCOPES`
<br>--
3. `data_url`
<br>Necessary if `csv_in` is `False`. 
<br>The ID of the raw creative testing data
<br>--
4. `data_sheet_name`
<br>Necessary if `csv_in` is `False`.<br>Where the data is located (e.g. 'Data' or 'Data!A1:B10')
<br>--
5. `results_url`
<br>Necessary if `csv_out` is `False`. 
<br>The ID of the spreadsheet to output the results to. (Can be the same spreadsheet with the raw data)
<br>--
6. `results_sheet_name`
<br>Necessary if `csv_out` is `False`. 
<br>Name of sheet/tab to output the results to. 
<br>If tab does not exist, will be created.

In [54]:
#data_url = 'https://docs.google.com/spreadsheets/d/1Yr-jmDKJixCcRRBK73yaWU-VtxZlcW5bnwm8wHqm538/edit#gid=1662096454'
#data_sheet_name = 'Data'
#results_url = 'https://docs.google.com/spreadsheets/d/197mB18nNN8LC9vLp0kIKgbTaBkVJQbPEDWfLpgDsWVQ/edit#gid=370640894'
#results_sheet_name = 'Creative Testing'

In [55]:
keys = 'credentials.json'
SCOPES = ['https://www.googleapis.com/auth/drive']

### Initialize necessary strings for csv
__Necessary strings__ if either `csv_in` or `csv_out` is `True`.

In [56]:
csv_path = 'test.csv'
csv_out_path = 'results.csv'

### Authenticate and Initialize Manager to work with Google Sheets
You can run this with no consequence even if you opt to use csv only.
<br> But you __must__ run this if there is gsheet use. There might be a pop-up window asking for authorization. Authorize.

In [57]:
if (not csv_in) or (not csv_out):
    authenticator = Authenticator(keys)
    creds = authenticator.get_creds(SCOPES)
    manager = SheetManager(creds)
    
if (not csv_in):
    data_spreadsheetId = re.findall(r"/spreadsheets/d/([a-zA-Z0-9-_]+)",data_url)[0]
    data_data_range = data_sheet_name
if (not csv_out):
    results_spreadsheetId = re.findall(r"/spreadsheets/d/([a-zA-Z0-9-_]+)",results_url)[0]
    results_data_range = results_sheet_name

### Load in Raw Data as DataFrame
Run this to load in the data.

In [58]:
if not csv_in:
    data_df = manager.get_values(spreadsheetId=data_spreadsheetId,
                            data_range=data_data_range)
else:
    data_df = pd.read_csv(csv_path)

### Define questions here:
1. `group_question`: `str`
<br>Indicate the question/variable that determine's the respondent's group (control or exposed, etc.)
<br>--
2. `control_value`: `str`
<br>Indicate the __value__ in the above question/variable that identifies a respondent to be in the __control__ group
<br>--
3. `weights_variable_name`: `str` or `None`
<br>Indicate the question/variable for the weights. __If no weights variable__, use `None`.


In [59]:
group_question = 'Q140'
control_value = '0' # string value, even for numbers
weights_variable_name = 'Default Weights' # or None

__Run this if no weights variable:__

In [60]:
if weights_variable_name is None:
    weights_variable_name = 'Count'
    data_df[weights_variable_name] = 1

### Recoding

#### IMPORTANT SECTION

1. `demo_question_list`: `list` (optional)
<br>Indicate demo questions. These questions may be used later on for cuts but will not go through significance testing.
<br> e.g. no use testing Gender and saying that exposure to the ad increase number of Females
2. `desired_dic`: `dict` __(mandatory)__
<br> __PLEASE FOLLOW THE FORMAT__
<br> Create a dictionary indicating the __desired response(s)__ for each __question__
<br> Only questions indicated in this dictionary will be tested and appear in the results.

In [61]:
######
# demo questions
# TO DO: auto cut based on demo in this list
# make it optional
######
demo_question_list = [
    'Q122',
]

### Follow the format of
"""
{
 'question_id': [desired option 1, desired option 2(, desired option 3, ...)],
 'question_id': [desired option 1, desired option 2(, desired option 3, ...)]
}
"""

### TODO: take into account "Not Shown" answer options (N/A) answer options

desired_dic = {
    'Q132.1': [2],
    
}


### Recode based on the above dictionary
Just run this. 
<br>Recodes values to `1` (desired) or `0` (not desired) based on the dictionary defined above.

In [62]:
def recode_desired(series):
    series = series.where(series.isin(desired_dic[series.name]), 'X')
    series = series.mask(series.isin(desired_dic[series.name]), 1)
    series[series=='X'] = 0
    series = pd.to_numeric(series)
    return(series)

test_questions = list(desired_dic.keys())

recoded_df = data_df.copy()
recoded_df[test_questions] = recoded_df[test_questions].apply(recode_desired, axis=0)
recoded_df

,Default Weights,Q121,Q122,Q128,Q129,Q130,Q131,Q132.1,Q132.2,Q132.3,...,Q186,Q187,Q188,Q189,Q190,Q191,Q192,Q193,Q194,Q195
0,0.771605,15,1,0,2,0,0,1,1,1,...,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.488095,18,0,1,3,1,2,0,1,1,...,2,分からない,ない,1.0,1.0,1.0,1.0,0.0,1.0,1.0
2,0.744048,17,1,4,0,1,2,0,2,2,...,1,なりたい自分になる,具体的なことがわからない,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.077586,25,1,7,1,0,0,0,1,2,...,2,料理,落ち着かない,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.644737,16,0,2,1,1,0,0,1,1,...,2,特にない,特にない,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.358696,25,1,4,2,1,3,1,1,1,...,2,飯テロ動画のYouTuberの紹介,嫌いではないが、見ていてお腹がすきます,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.059322,27,0,4,0,1,2,0,2,1,...,2,メイクの方法。,メイクの、クローズアップ,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,0.905797,21,1,2,2,0,3,1,1,2,...,2,わかりません。,ユーチューバー,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,0.905797,21,1,3,2,0,0,0,1,2,...,1,特になし,嫌いな点はない,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,0.672043,20,1,2,2,1,1,1,2,2,...,1,器具を使わずにトレーニングできる,なし,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### Generate overall cut
Run this to at least get the overall cut.

In [63]:
overall_df = recoded_df.copy()

cuts_dic = {
    'Overall': overall_df,
}

In [64]:
for demo_question in demo_question_list:
    for cut in overall_df[demo_question].unique().tolist():
        cuts_dic[demo_question+': '+str(cut)] = overall_df[overall_df[demo_question]==cut]

### Additional Cuts (optional)
1. Identify the cuts you want, and save them as Data Frames.
2. Then, add on to the `cuts_dic` they key and the Data Frame.
<br> e.g. `cuts_dic['Male']= male_df`

In [65]:
cuts_dic.keys()

dict_keys(['Overall', 'Q122: 1', 'Q122: 0'])

### Execute Tests

Just run this.
Execute the tests on all the cuts indicated above, for the questions that have been selected for testing.

In [66]:
tables = []
control_value = str(control_value)
for cut in cuts_dic.keys():    
    table = cuts_dic[cut].copy()
    table[group_question] = table[group_question].astype(str)
    weights_table = table[[group_question, weights_variable_name]]
    bases = weights_table.groupby(table[group_question]).sum()[weights_variable_name]
    questions = test_questions
    table = pd.pivot_table(table, values=questions, columns=group_question, aggfunc='sum')
    initial_cols = table.columns
    for group in table.columns:
        table[group+' Base'] = bases[group]
        table[group+' Base'] = table[group+' Base'].apply(np.round, decimals=(results_dp))
        table[group+' Desired %'] = table[group] / table[group+' Base']
        table[group+' Desired %'] = table[group+' Desired %'].apply(np.round, decimals=(results_percentage_dp))
        if group != control_value:
            table[group+' Abs Lift %'] = table[group+' Desired %'] - table[control_value+' Desired %']
            table[group+' Abs Lift %'] = table[group+' Abs Lift %'].apply(np.round, decimals=(results_percentage_dp))
            table[group+' p-value'] = table.apply(lambda x: ztest(
                                                                [x[group], x[control_value]],
                                                                [x[group+' Base'], x[control_value+' Base']]
                                                                    )[1], axis=1)
            table[group+' Significance'] = 'No Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .2) & (table[group+' Abs Lift %'] > 0)] = 'Directional Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .1) & (table[group+' Abs Lift %'] > 0)] = 'Significant Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .05) & (table[group+' Abs Lift %'] > 0)] = 'Strong Significant Lift'
            table[group+' Significance'].loc[(table[group+' p-value'] < .1) & (table[group+' Abs Lift %'] < 0)] = 'Negative Lift'

    suffixes = [' Base', ' Desired %', ' Abs Lift %', ' Significance'] # can include control_name+' p-value'
    table['Cut'] = cut
    col_order = ['Cut', control_value, control_value+' Base', control_value+' Desired %']
    for col in initial_cols:
        if col != control_value:
            col_order.append(col)
            for suffix in suffixes:
                col_order.append(col+suffix)
    table = table[col_order]
    #table = table.rename(index=codebook_dic)
    tables.append(table)
    
results = pd.concat(tables, axis=0)

In [67]:
print(results)

Q140        Cut    0  0 Base  0 Desired %    1  1 Base  1 Desired %  \
Q132.1  Overall  138   250.0         0.55  152   250.0         0.61   
Q132.1  Q122: 1   91   125.0         0.73  103   125.0         0.82   
Q132.1  Q122: 0   47   125.0         0.38   49   125.0         0.39   

Q140    1 Abs Lift %    1 Significance    2  ...    8  8 Base  8 Desired %  \
Q132.1          0.06           No Lift  144  ...  151   250.0         0.60   
Q132.1          0.09  Significant Lift  105  ...  101   125.0         0.81   
Q132.1          0.01           No Lift   39  ...   50   125.0         0.40   

Q140   8 Abs Lift %    8 Significance    9  9 Base  9 Desired % 9 Abs Lift %  \
Q132.1         0.05           No Lift  170   250.0         0.68         0.13   
Q132.1         0.08  Directional Lift  108   125.0         0.86         0.13   
Q132.1         0.02           No Lift   62   125.0         0.50         0.12   

Q140             9 Significance  
Q132.1  Strong Significant Lift  
Q132.1  Stron

### Write results automatically to gsheets, or output a csv. 
Based on selected option at the top.

In [68]:
if not csv_out:
    results_to_write = results.reset_index()

    values = []
    values.append(results_to_write.columns.tolist())
    for row in results_to_write.values.tolist():
        values.append(row)

    if results_data_range not in manager.get_existing_sheets_names(spreadsheetId=results_spreadsheetId):
        manager.add_sheet(spreadsheetId=results_spreadsheetId, sheet_name=results_data_range)

    manager.update_values(spreadsheetId=results_spreadsheetId,
                          update_range=results_data_range,
                          values=values)
else:
    results.to_csv(index=True, header=True)